In [1]:
import os
import sys

# Obtener el directorio actual de la notebook
notebook_dir = os.getcwd()

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Agregar el directorio padre al path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from libs.config.config_variables import MAX_WORKERS, CATALOG_CSV_DIR

from modules.catalog_scraper.seismic_scraper import SeismicScraper

# 0. Configuración básica de catálogos

In [3]:
# Límites geográficos (Perú y alrededores)
MIN_LAT, MAX_LAT = -20.0, 0.0
MIN_LON, MAX_LON = -85.0, -68.0

# Año de inicio del scraping
START_YEAR = 1400

In [4]:
# Registrar los catálogos csv
historical_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, csv_base_path=CATALOG_CSV_DIR
)
results = historical_scraper.scrape_events_from_csv(
    catalogs=["gcmt", "isc-gem", "sara"],

    # start_year=START_YEAR,
    # min_lat=MIN_LAT,
    # max_lat=MAX_LAT,
    # min_lon=MIN_LON,
    # max_lon=MAX_LON,
)


2025-11-02 17:20:56,740 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events_from_csv
2025-11-02 17:20:56,740 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 1 archivos CSV para GCMT
2025-11-02 17:20:56,745 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para ISC-GEM
2025-11-02 17:20:56,747 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para SARA
2025-11-02 17:20:56,748 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: 5 archivos con 40 workers
2025-11-02 17:20:56,750 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: GCMT - gcmt_data.csv
2025-11-02 17:20:56,750 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-cat.csv
2025-11-02 17:20:56,774 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-suppl.csv
2025-11-02 17:20:56,775 [INFO] modules.ca

In [5]:
historical_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|     178748      | 178748 |      0       |    1    |
+-----------------+--------+--------------+---------+


# 1. Ejecución de scraping de catálogos


In [6]:
recent_data_scraper = SeismicScraper(max_workers=MAX_WORKERS)

results = recent_data_scraper.scrape_events(
    start_year=START_YEAR,
    min_lat=MIN_LAT,
    max_lat=MAX_LAT,
    min_lon=MIN_LON,
    max_lon=MAX_LON,
    catalogs=["usgs", "isc", "igp"],
)

2025-11-02 17:21:08,102 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events
2025-11-02 17:21:08,103 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: 1878 tareas con 40 workers
2025-11-02 17:21:08,104 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1400
2025-11-02 17:21:08,105 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1401
2025-11-02 17:21:08,105 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1402
2025-11-02 17:21:08,129 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1403
2025-11-02 17:21:08,131 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1404
2025-11-02 17:21:08,144 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1405
2025-11-02 17:21:08,145 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 1406
2025-11-02 17:21:08,145 [INFO]

In [7]:
recent_data_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|      19324      | 19322  |      2       |   615   |
+-----------------+--------+--------------+---------+
